# Example for `vista`?

This is sort of a minimum working example, to see if we can replace the VTK plotting currently included in `GemPy` with `vista`. It is a strip-down and modified version of the example given in [Chapter 1.1](https://nbviewer.jupyter.org/github/cgre-aachen/gempy/blob/master/notebooks/tutorials/ch1-1_Basics.ipynb).

All the VTK stuff in `GemPy` is located in `gempy/plot/visualization_3d.py` in the classes `vtkVisualization` and `GemPyvtkInteract`.

In [1]:
# Importing GemPy
import gempy as gp

# Importing auxiliary libraries
import numpy as np
import matplotlib.pyplot as plt

## Importing and creating a set of input data

In [2]:
# Create a geo-model
geo_model = gp.create_model('GemPy-vista')

# Importing the data from CSV-files and setting extent and resolution
gp.init_data(
    geo_model,
    [0,2000., 0,2000., -2000, 0.],[50, 50, 50],
    path_o = "data/simple_fault_model_orientations.csv",
    path_i = "data/simple_fault_model_points.csv",
    default_values=True
)

GemPy-vista  2019-05-10 11:35

Map the different series

In [3]:
gp.map_series_to_surfaces(
    geo_model,
    {
        "Fault_Series": 'fault',
        "Strat_Series": ('seawater', 'overburden', 'target', 'underburden', 'basement')
    },
    remove_unused_series=True
)

,surface,series,order_surfaces,isBasement,color,id
4,fault,Fault_Series,1,False,#443988,1
0,target,Strat_Series,1,False,#015482,2
1,underburden,Strat_Series,2,False,#9f0052,3
2,overburden,Strat_Series,3,False,#ffbe00,4
3,seawater,Strat_Series,4,False,#728f02,5
5,basement,Strat_Series,5,True,#ff3f20,6


Set the fault series to fault

In [4]:
geo_model.set_is_fault(['Fault_Series'])

Fault colors changed. If you do not like this behavior, set change_color to False.


,isFault,isFinite
Fault_Series,True,False
Strat_Series,False,False


## A note about the grid that is used

`geo_model.grid` is the `GemPy`-equivalent to `grid.gridCC` in discretize.

### Important difference: `GemPy` uses C-sorting, while `discretize` uses F-sorting.

In [5]:
geo_model.grid

Grid Object. Values: 
array([[   20.,    20., -1980.],
       [   20.,    20., -1940.],
       [   20.,    20., -1900.],
       ...,
       [ 1980.,  1980.,  -100.],
       [ 1980.,  1980.,   -60.],
       [ 1980.,  1980.,   -20.]])

## Model generation

In [6]:
gp.set_interpolation_data(geo_model,
                          compile_theano=True,
                          theano_optimizer='fast_compile',
                          verbose=[])


Compiling theano function...
Compilation Done!


### Calculate the solution

In [7]:
sol = gp.compute_model(geo_model, compute_mesh=True)

## Plot it with VTK

In [8]:
gp.plot.plot_3D(geo_model)

closing vtk


Using the rescaled interpolation data, **we can also run our 3D VTK visualization in an interactive mode** which allows us to **alter and update our model in real time**. Similarly to the interactive 3D visualization of our input data, the changes are permamently saved (in the InterpolationInput dataframe object). Addtionally, the resulting changes in the geological models are re-computed in real time.